In [ ]:
import pandas as pd

In [ ]:
# get train csv data
df = pd.read_csv("../data/raw/train.csv")
df.head()

In [ ]:
# x, y split
X = df.drop(columns='forward_returns')
y = df['forward_returns']
X.shape, y.shape

In [ ]:
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('src'))))
from src.cv import PurgedWalkForwardCV

cv_split = PurgedWalkForwardCV(n_splits=5, 
                         embargo=20, 
                         purge=True,
                         purge_period=5,
                         train_ratio=0.8)

split_df = []
for train_idx, val_idx in cv_split.split(X, y):
    print(train_idx, val_idx)
    
    X_train = X.iloc[train_idx]
    y_train = y.iloc[train_idx]

    X_valid = X.iloc[val_idx]
    y_valid = y.iloc[val_idx]

    split_df.append((X_train, y_train, X_valid, y_valid))

In [ ]:
from pathlib import Path
from lightgbm import LGBMRegressor
from src.metric import CompetitionMetric
from src.position import PositionOptimizer, create_position_mapper
from src.risk import RiskLabeler
from src.utils import load_config
import numpy as np

project_root = Path.cwd()
config_path = project_root / "conf/params.yaml"
if not config_path.exists():
    for parent in project_root.parents:
        candidate = parent / "conf/params.yaml"
        if candidate.exists():
            config_path = candidate
            break
if not config_path.exists():
    raise FileNotFoundError(f"conf/params.yaml not found from {project_root}")

config = load_config(str(config_path))
return_cfg = config.get("model_return", {}).get("lightgbm", {})
risk_cfg = config.get("risk", {}).get("lightgbm", {}).get("fixed_params", {})
return_model_params = {
    "n_estimators": return_cfg.get("n_estimators", 800),
    "learning_rate": return_cfg.get("learning_rate", 0.05),
    "num_leaves": return_cfg.get("num_leaves", 64),
    "feature_fraction": return_cfg.get("feature_fraction", 0.8),
    "bagging_fraction": return_cfg.get("bagging_fraction", 0.8),
    "bagging_freq": return_cfg.get("bagging_freq", 5),
    "min_child_samples": return_cfg.get("min_child_samples", 20),
    "max_depth": return_cfg.get("max_depth", -1),
    "objective": return_cfg.get("objective", "regression"),
    "random_state": return_cfg.get("random_state", 42),
    "verbosity": return_cfg.get("verbosity", -1),
    "n_jobs": -1
}
risk_model_params = {
    "n_estimators": return_cfg.get("n_estimators", 600),
    "learning_rate": return_cfg.get("learning_rate", 0.05),
    "num_leaves": risk_cfg.get("num_leaves", 48),
    "feature_fraction": 0.7,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "min_child_samples": risk_cfg.get("min_data_in_leaf", 15),
    "reg_alpha": risk_cfg.get("lambda_l1", 0.1),
    "reg_lambda": risk_cfg.get("lambda_l2", 0.1),
    "objective": "regression",
    "random_state": risk_cfg.get("random_state", 42),
    "verbosity": risk_cfg.get("verbosity", -1),
    "n_jobs": -1
}
risk_labeler = RiskLabeler(window=config.get("risk", {}).get("label", {}).get("window", 20))
metric_calc = CompetitionMetric()
fold_results = []

for fold_idx, (X_train, y_train, X_valid, y_valid) in enumerate(split_df, start=1):
    print(X_train.shape, y_train.shape)
    print(X_valid.shape, y_valid.shape)

    # 1. Train Return Model on train fold
    return_model = LGBMRegressor(**return_model_params)
    return_model.fit(X_train, y_train)
    train_r_hat = return_model.predict(X_train)

    # 2. Train Risk Model on train fold
    risk_train_df = pd.concat([X_train, y_train], axis=1).sort_values("date_id").copy()
    risk_train_df["risk_label"] = risk_labeler.create_labels(risk_train_df, "forward_returns")
    risk_targets = risk_train_df["risk_label"].bfill().ffill()
    if risk_targets.isna().any():
        fallback = risk_train_df["forward_returns"].rolling(window=20, min_periods=5).std()
        risk_targets = risk_targets.fillna(fallback).fillna(risk_train_df["forward_returns"].abs())
    risk_model = LGBMRegressor(**risk_model_params)
    risk_model.fit(risk_train_df[X_train.columns], risk_targets)
    train_sigma_hat = risk_model.predict(X_train)

    # 3. Optimize Position Strategy on train fold
    mapper = create_position_mapper(strategy="sharpe_scaling")
    optimizer = PositionOptimizer(mapper)
    optimal_params = optimizer.optimize_sharpe_params(
        r_hat=train_r_hat,
        sigma_hat=train_sigma_hat,
        actual_returns=y_train.values
    )

    # 4. Make predictions on validation fold
    valid_r_hat = return_model.predict(X_valid)
    valid_sigma_hat = risk_model.predict(X_valid)

    # 5. Convert these predictions to positions
    valid_positions = mapper.map_positions(
        r_hat=valid_r_hat,
        sigma_hat=valid_sigma_hat,
        k=optimal_params["k"],
        b=optimal_params["b"]
    )

    # 6. Evaluate strategy Sharpe on validation fold
    fold_score = metric_calc.calculate_score(
        allocations=valid_positions,
        forward_returns=y_valid.values
    )
    fold_results.append(fold_score)
    print(
        f"Fold {fold_idx} → score: {fold_score['score']:.4f}, "
        f"sharpe: {fold_score['sharpe']:.4f}, k={optimal_params['k']:.3f}, b={optimal_params['b']:.3f}"
    )

if fold_results:
    mean_score = np.mean([res["score"] for res in fold_results])
    mean_sharpe = np.mean([res["sharpe"] for res in fold_results])
    print(f"Average validation score: {mean_score:.4f}, Average sharpe: {mean_sharpe:.4f}")
